In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
from sklearn.model_selection import train_test_split
import dama as dm
import pickle
import os

from freedom.toy_model.toy_model_functions import toy_model
from freedom.toy_model.detectors import get_box_detector
from types import SimpleNamespace

%load_ext autoreload
%autoreload 2

In [ ]:
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14 
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['legend.fontsize'] = 14

In [ ]:
geo = np.load('../../freedom/resources/geo_array_upgrade.npy')
str_pos = geo[:, 42][:, :2]
z_pos = np.linspace(-200, -500, 20) #80

detector = np.append(np.repeat(str_pos, len(z_pos), axis=0), np.tile(z_pos, len(str_pos)).reshape(-1,1), axis=1)

#fig = plt.figure()
#ax = fig.add_subplot(projection='3d')
#ax.scatter(detector[:,0], detector[:,1], detector[:,2])

In [ ]:
toy_experiment = toy_model(detector)

In [ ]:
truth = np.array([50., -50., -250., 0, 0, np.arccos(0), 20., 10.])

In [ ]:
# generate one test event
test_event = toy_experiment.generate_event(truth)

In [ ]:
test_event

In [ ]:
%%time
# Grid scan

g = dm.GridData(x=np.linspace(0, 100, 100), y=np.linspace(-100, 0, 100))

g['dom_hit_term'] = np.empty(g.shape)
g['dom_charge_terms'] = np.empty(g.shape)
g['total_charge_hit_terms'] = np.empty(g.shape)
g['total_charge_terms'] = np.empty(g.shape)

p = np.copy(truth)

for idx in np.ndindex(g.shape):
    p[0] = g['x'][idx]
    p[1] = g['y'][idx]
    segments = toy_experiment.model(*p)
    g['dom_hit_term'][idx] = toy_experiment.nllh_p_term_dom(segments, test_event[0])
    g['dom_charge_terms'][idx] = toy_experiment.nllh_N_term_dom(segments, test_event[1])
    g['total_charge_hit_terms'][idx] = toy_experiment.nllh_p_term_tot(segments, test_event[0])
    g['total_charge_terms'][idx] = toy_experiment.nllh_N_term_tot(segments, test_event[1])

In [ ]:
g['dom_hit_term'] -= g['dom_hit_term'].min()
g['dom_charge_terms'] -= g['dom_charge_terms'].min()
g['dom_llh'] = g['dom_hit_term'] + g['dom_charge_terms']
g['total_charge_hit_terms'] -= g['total_charge_hit_terms'].min()
g['total_charge_terms'] -= g['total_charge_terms'].min()
g['total_charge_llh'] = g['total_charge_hit_terms'] + g['total_charge_terms']
g['dom_llh'] -= g['dom_llh'].min()
g['total_charge_llh'] -= g['total_charge_llh'].min()

In [ ]:
def plot_truth(axes, truth):
    if not isinstance(axes, np.ndarray):
        axes = np.array([axes])
    for ax in axes.flatten():
        ax.plot([truth[0]], [truth[1]], marker='$T$', markersize=10, color='k')

In [ ]:
def plot_diff(a, b, axes, title_a='a', title_b='b', vmax=None, limit_diff=False, **kwargs):
    
    levels = stats.chi2(df=2).isf(stats.norm.sf(np.arange(1,6))*2)/2    
    labels = [str(i) + r'$\sigma$' for i in range(1,6)]
    colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red']
    colors = plt.cm.viridis(np.linspace(0, 0.9, 6))
    #a.plot(ax=axes[0], cmap='Greys', label=r'$\Delta LLH$', **kwargs)
    a.plot_contour(ax=axes[0], levels=levels, labels=labels, colors=colors, label=r'$\Delta LLH$', **kwargs)
    axes[0].set_title(title_a)
    #b.plot(ax=axes[1], cmap='Greys', label=r'$\Delta LLH$', **kwargs)
    b.plot_contour(ax=axes[1], levels=levels,  labels=labels, colors=colors, label=r'$\Delta LLH$', **kwargs)
    axes[1].set_title(title_b)
    diff = a - b
    if limit_diff:
        diff.plot(ax=axes[2], cmap='RdBu', cbar=True, vmin=-vmax, vmax=vmax, label=r'$\Delta LLH$', **kwargs)
        #diff.plot_contour(ax=axes[2], levels=levels, labels=labels, colors=colors, label=r'$\Delta LLH$', **kwargs)
        #np.clip(-diff, 0, None).plot_contour(ax=axes[2], levels=[0.1,0.2, 0.3], colors=['r']*2)
    else:
        diff.plot(ax=axes[2], cmap='RdBu', cbar=True, vmin=-np.max(np.abs(diff)), vmax=np.max(np.abs(diff)), label=r'$\Delta LLH$', **kwargs) 
        #diff.plot_contour(ax=axes[2], levels=levels, labels=labels, colors=colors, label=r'$\Delta LLH$', **kwargs)
    axes[2].set_title(title_a + ' - ' + title_b)

In [ ]:
#stats.norm.isf(stats.chi2(df=2).sf(g['dom_llh']*2)/2)

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(20,17))
plt.subplots_adjust(wspace=0.3, hspace=0.3)

plot_diff(g['dom_hit_term'], g['total_charge_hit_terms'], axes=ax[0], title_a='per DOM hit', title_b='total hit', vmax=20, limit_diff=True)
plot_diff(g['dom_charge_terms'], g['total_charge_terms'], axes=ax[1], title_a='per DOM charge', title_b='total charge', vmax=20, limit_diff=True)
plot_diff(g['dom_llh'], g['total_charge_llh'], axes=ax[2], title_a='per DOM llh', title_b='total llh', limit_diff=False)

plot_truth(ax, truth)

In [ ]:
stats.chi2(df=2).isf(stats.norm.sf(np.arange(1,6))*2)/2

# Train NNs

In [ ]:
from freedom.toy_model import NNs
%aimport freedom.toy_model.NNs

In [ ]:
events, truths = toy_experiment.generate_event_box(n=100_000, x_lim=(4,100), y_lim=(-94,-21), z_lim=(-510,-210),
                                                   e_lim=(5,60), t_width=0, contained=False, N_min=3)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
nGPUs = strategy.num_replicas_in_sync

## Hit Net - per dom

In [ ]:
x, t = NNs.get_hit_data(events, truths)
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.1, random_state=42)
d_train = NNs.DataGenerator(x_train, t_train, batch_size=4096*nGPUs, shuffle='inDOM', time_spread=150)
d_valid = NNs.DataGenerator(x_test, t_test, batch_size=4096*nGPUs, shuffle='inDOM', time_spread=150)

In [ ]:
optimizer = tf.keras.optimizers.Adam(2e-5)

with strategy.scope():
    hmodel = NNs.get_hmodel(x_shape=6, t_shape=8, trafo=NNs.hit_trafo_3D, activation='swish', final_activation='swish')
    hmodel.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
hist = hmodel.fit(d_train, epochs=15, verbose=1, validation_data=d_valid)

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.yscale('log')

In [ ]:
inp, lab = d_valid.__getitem__(0)
pred = hmodel.predict(inp, batch_size=4096).flatten()
plt.hist(pred[lab==0], 100, histtype='step')
plt.hist(pred[lab==1], 100, histtype='step');
plt.yscale('log')

In [ ]:
xxs = np.repeat(test_event[0][np.newaxis, :], np.prod(g.shape), axis=0)
xxs = xxs.reshape(-1, 6)

tts = np.repeat(truth[np.newaxis, :], np.prod(g.shape), axis=0)
tts[:, 0] = g.get_array('x', flat=True)
tts[:, 1] = g.get_array('y', flat=True)
tts = np.repeat(tts, len(test_event[0]), axis=0)

hmodel.layers[-1].activation = tf.keras.activations.linear
hmodel.compile()

llhs = -hmodel.predict((xxs, tts), batch_size=4096)

llhs = np.sum(np.nan_to_num(llhs.reshape(-1, len(test_event[0]))), axis=1)

g.hit_llh_dom = llhs.reshape(g.shape)

g.hit_llh_dom -= g.hit_llh_dom.min()

fig, ax = plt.subplots(1, 3, figsize=(20,5))
plt.subplots_adjust(wspace=0.3)

plot_diff(g.dom_hit_term, g.hit_llh_dom, title_a='Analytic', title_b='NN', vmax=10, axes=ax, limit_diff=True)
plot_truth(ax, truth)
#plt.savefig('images/hitNNtest.png', bbox_inches='tight')

In [ ]:
#hmodel.save('networks/upgrade_toy_hitnet_dom.h5')
#hmodel = tf.keras.models.load_model('networks/upgrade_toy_hitnet_dom.h5',
#                                     custom_objects={'hit_trafo_3D':NNs.hit_trafo_3D})

## Charge Net - per DOM

In [ ]:
def remove_0s(x, t, n=1):
    keep = np.where(x[:,-1]>0)[0]
    keep = np.append(keep, np.random.choice(np.where(x[:,-1]==0)[0], n*len(keep)))
    return x[keep], t[keep]

In [ ]:
x, t = NNs.get_dom_data(events, truths, detector)
#x, t = remove_0s(x, t)
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.1, random_state=42)

d_train = NNs.DataGenerator(x_train, t_train, batch_size=4096*nGPUs)
d_valid = NNs.DataGenerator(x_test, t_test, batch_size=4096*nGPUs)

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-3)

with strategy.scope():
    dmodel = NNs.get_hmodel(x_shape=4, t_shape=8, trafo=NNs.dom_trafo_3D, activation='swish', final_activation='swish')
    dmodel.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
hist = dmodel.fit(d_train, epochs=10, verbose=1, validation_data=d_valid)

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.yscale('log')

In [ ]:
inp, lab = d_valid.__getitem__(0)
pred = dmodel.predict(inp, batch_size=4096).flatten()
plt.hist(pred[lab==0], 100, histtype='step')
plt.hist(pred[lab==1], 100, histtype='step');
plt.yscale('log')

In [ ]:
xx = []
ind = test_event[0][:, 5]
for i in range(len(detector)):
    d = np.append(detector[i], np.sum(ind==i))
    xx.append(list(d))
xxs = np.repeat(np.array(xx)[np.newaxis, :], np.prod(g.shape), axis=0)
xxs = xxs.reshape(-1, 4)

tts = np.repeat(truth[np.newaxis, :], np.prod(g.shape), axis=0)
tts[:, 0] = g.get_array('x', flat=True)
tts[:, 1] = g.get_array('y', flat=True)
tts = np.repeat(tts, len(detector), axis=0)

dmodel.layers[-1].activation = tf.keras.activations.linear
dmodel.compile()

llhs = -dmodel.predict((xxs, tts), batch_size=4096)

llhs = np.sum(np.nan_to_num(llhs.reshape(-1, len(detector))), axis=1)

g.charge_llh_dom = llhs.reshape(g.shape)

g.charge_llh_dom -= g.charge_llh_dom.min()

fig, ax = plt.subplots(1, 3, figsize=(20,5))
plt.subplots_adjust(wspace=0.3)

plot_diff(g.dom_charge_terms, g.charge_llh_dom, title_a='Analytic', title_b='NN', vmax=10, axes=ax, limit_diff=True)
plot_truth(ax, truth)
#plt.savefig('images/hitNNtest.png', bbox_inches='tight')

In [ ]:
#dmodel.save('networks/upgrade_toy_chargenet_dom.h5')
#dmodel = tf.keras.models.load_model('networks/upgrade_toy_chargenet_dom.h5',
#                                     custom_objects={'dom_trafo_3D':NNs.dom_trafo_3D})

## LLH - per DOM

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(20,17))
plt.subplots_adjust(wspace=0.25, hspace=0.25)

plot_diff(g.dom_hit_term, 
          g.hit_llh_dom, 
          title_a='Hit Analytic', title_b='Hit NN', vmax=10, axes=ax[0], limit_diff=True)
plot_truth(ax, truth)

plot_diff(g.dom_charge_terms, 
          g.charge_llh_dom, 
          title_a='Charge Analytic', title_b='Charge NN', vmax=10, axes=ax[1], limit_diff=True)
plot_truth(ax, truth)

ana, NN = g.dom_hit_term+g.dom_charge_terms, g.hit_llh_dom+g.charge_llh_dom
plot_diff(ana-ana.min(), 
          NN-NN.min(), 
          title_a='Analytic', title_b='NN', vmax=10, axes=ax[2], limit_diff=True)
plot_truth(ax, truth)

#plt.savefig('images/NNtest_perDOM.png', bbox_inches='tight')

energy

In [ ]:
%%time

g2 = dm.GridData(x=np.linspace(5, 50, 100), y=np.linspace(5, 50, 100))

g2['dom_hit_term'] = np.empty(g.shape)
g2['dom_charge_terms'] = np.empty(g.shape)

p = np.copy(truth)

for idx in np.ndindex(g2.shape):
    p[6] = g2['x'][idx]
    p[7] = g2['y'][idx]
    segments = toy_experiment.model(*p)
    g2['dom_hit_term'][idx] = toy_experiment.nllh_p_term_dom(segments, test_event[0])
    g2['dom_charge_terms'][idx] = toy_experiment.nllh_N_term_dom(segments, test_event[1])
    
g2['dom_hit_term'] -= g2['dom_hit_term'].min()
g2['dom_charge_terms'] -= g2['dom_charge_terms'].min()
g2['dom_llh'] = g2['dom_hit_term'] + g2['dom_charge_terms']
g2['dom_llh'] -= g2['dom_llh'].min()

In [ ]:
xxs = np.repeat(test_event[0][np.newaxis, :], np.prod(g2.shape), axis=0)
xxs = xxs.reshape(-1, 6)

tts = np.repeat(truth[np.newaxis, :], np.prod(g2.shape), axis=0)
tts[:, 6] = g2.get_array('x', flat=True)
tts[:, 7] = g2.get_array('y', flat=True)
tts = np.repeat(tts, len(test_event[0]), axis=0)

hmodel.layers[-1].activation = tf.keras.activations.linear
hmodel.compile()

llhs = -hmodel.predict((xxs, tts), batch_size=4096)
llhs = np.sum(np.nan_to_num(llhs.reshape(-1, len(test_event[0]))), axis=1)

g2.hit_llh_dom = llhs.reshape(g2.shape)
g2.hit_llh_dom -= g2.hit_llh_dom.min()


xx = []
ind = test_event[0][:, 5]
for i in range(len(detector)):
    d = np.append(detector[i], np.sum(ind==i))
    xx.append(list(d))
xxs = np.repeat(np.array(xx)[np.newaxis, :], np.prod(g2.shape), axis=0)
xxs = xxs.reshape(-1, 4)

tts = np.repeat(truth[np.newaxis, :], np.prod(g2.shape), axis=0)
tts[:, 6] = g2.get_array('x', flat=True)
tts[:, 7] = g2.get_array('y', flat=True)
tts = np.repeat(tts, len(detector), axis=0)

dmodel.layers[-1].activation = tf.keras.activations.linear
dmodel.compile()

llhs = -dmodel.predict((xxs, tts), batch_size=4096)
llhs = np.sum(np.nan_to_num(llhs.reshape(-1, len(detector))), axis=1)

g2.charge_llh_dom = llhs.reshape(g2.shape)
g2.charge_llh_dom -= g2.charge_llh_dom.min()

In [ ]:
levels = stats.chi2(df=2).isf(stats.norm.sf(np.arange(1,6))*2)/2    
labels = [str(i) + r'$\sigma$' for i in range(1,6)]
colors = plt.cm.viridis(np.linspace(0, 0.9, 6))

fig, ax = plt.subplots(2, 2, figsize=(10,9))
plt.subplots_adjust(wspace=0.25, hspace=0.25)

g2.dom_hit_term.plot_contour(ax=ax[0][0], levels=levels, labels=labels, colors=colors, label=r'$\Delta LLH$')
ax[0][0].plot([truth[6]], [truth[7]], marker='$T$', markersize=10, color='k')

g2.hit_llh_dom.plot_contour(ax=ax[0][1], levels=levels, labels=labels, colors=colors, label=r'$\Delta LLH$')
ax[0][1].plot([truth[6]], [truth[7]], marker='$T$', markersize=10, color='k')

g2.dom_charge_terms.plot_contour(ax=ax[1][0], levels=levels, labels=labels, colors=colors, label=r'$\Delta LLH$')
ax[1][0].plot([truth[6]], [truth[7]], marker='$T$', markersize=10, color='k')

g2.charge_llh_dom.plot_contour(ax=ax[1][1], levels=levels, labels=labels, colors=colors, label=r'$\Delta LLH$')
ax[1][1].plot([truth[6]], [truth[7]], marker='$T$', markersize=10, color='k')

# Reco

In [ ]:
from spherical_opt import spherical_opt
from multiprocessing import Pool, Process

In [ ]:
bounds = np.array([[14,90], [-84,-31], [-500,-200], [-700,700], [0,2*np.pi], [0,np.pi], [5,50], [5,50]])

def init_points(hits, n_live_points, bound=bounds, seed=[None]):
    if seed[0] == None:
        avg = np.average(hits[:, :4], axis=0)
        low_lims = np.concatenate([avg-np.array([10,10,30,200]), np.array([0,0,5,5])])
        hig_lims = np.concatenate([avg+np.array([10,10,30,0]), np.array([2*np.pi,np.pi,50,50])])
    else:
        low_lims = seed - np.array([1, 1, 1, 5, 0.5, 0.3, 3, 3])
        hig_lims = seed + np.array([1, 1, 1, 5, 0.5, 0.3, 3, 3])
    
    uniforms = np.random.uniform(size=(n_live_points, 8))
    initial_points = low_lims + uniforms * (hig_lims - low_lims)
    initial_points = np.clip(initial_points, bounds[:, 0], bounds[:, 1])
    return initial_points

def remove_strings(events, str_num, om_per_string=80):
    if isinstance(str_num, int):
        str_num = [str_num]
        
    remove_indx = []
    for s in str_num:
        remove_indx.extend(np.arange(om_per_string)+s*om_per_string)
        
    events_out = []
    for e in events:
        h, c = np.array(e[0]), np.array(e[1])
        if np.sum(c) == 0:
            events_out.append((h,c))
            continue

        h = h[[item not in remove_indx for item in h[:,-1]]]
        c = np.delete(c, remove_indx)
        events_out.append((h,c))
    
    return np.array(events_out), remove_indx

In [ ]:
events, truths = toy_experiment.generate_event_box(n=1000, x_lim=(14,90), y_lim=(-84,-31), z_lim=(-500,-200),
                                                   e_lim=(5,50), t_width=0, contained=False, N_min=3)

In [ ]:
events_red, remove_indx = remove_strings(events, [0,2])
toy_experiment_red = toy_model(np.delete(detector, remove_indx, axis=0))

### Analytic

In [ ]:
def LLH_ana(X, event, form='dom', fix=[None], bounds=bounds, toy_experiment=toy_experiment):
    if fix[0] != None:
        X = np.insert(X, fix[0], fix[1])
        
    if ~np.alltrue(np.logical_and(bounds[:,0] <= X, X <= bounds[:,1]), axis=-1):
        return 1e9
    
    segments = toy_experiment.model(*X)
    if form == 'dom':
        h_term = toy_experiment.nllh_p_term_dom(segments, event[0])
        c_term = toy_experiment.nllh_N_term_dom(segments, event[1])
    elif form == 'total':
        h_term = toy_experiment.nllh_p_term_tot(segments, event[0])
        c_term = toy_experiment.nllh_N_term_tot(segments, event[1])
    else:
        raise NameError("Formulation must be one of ['total', 'dom'], not "+form)
    
    return c_term + h_term

def fit_event_ana(event):
    event, truth = event
    
    def eval_LLH(params):
        if params.ndim == 1:
            return LLH_ana(params, event, toy_experiment=toy_experiment)
        else:
            llhs = []
            for p in params:
                llhs.append(LLH_ana(p, event, toy_experiment=toy_experiment))
            return np.array(llhs)

    # seeding
    initial_points = init_points(event[0], 97) #, seed=truth
    
    # free fit
    fit_res = spherical_opt.spherical_opt(
        func=eval_LLH,
        method="CRS2",
        initial_points=initial_points,
        rand=np.random.default_rng(42),
        spherical_indices=[[4,5]],
        batch_size=12,
    )

    return list(fit_res['x'])

In [ ]:
%%time
with Pool(10) as p:
    outs = p.map(fit_event_ana, zip(events, truths))
recos_ana = np.array(outs)

In [ ]:
diff_ana = recos_ana - truths

### NNs

In [ ]:
import math

from functools import partial
from freedom.llh_service.llh_service import LLHService
from freedom.llh_service.llh_client import LLHClient
from freedom.reco import crs_reco

In [ ]:
loc = 'networks/'
service_conf = {
        "poll_timeout": 1,
        "flush_period": 1,
        "n_hypo_params": 8,
        "n_hit_features": 6,
        "n_evt_features": len(detector)*4,
        "batch_size" : {
          "n_hypos": 200,
          "n_observations": 6000, 
        },
        "send_hwm": 10000,
        "recv_hwm": 10000,
        "hitnet_file": loc+'upgrade_toy_hitnet_dom.h5',
        "domnet_file": loc+'upgrade_toy_chargenet_dom.h5',
        "ndoms": len(detector),
        "toy": True
}

In [ ]:
n_gpus = 4

base_req = "ipc:///tmp/recotestreq"
base_ctrl = "ipc:///tmp/recotestctrl"

req_addrs = []
ctrl_addrs = []
for i in range(n_gpus):
    req_addrs.append(f'{base_req}{i}')
    ctrl_addrs.append(f'{base_ctrl}{i}')
    
procs = []
for i in range(n_gpus):
    proc = Process(target=crs_reco.start_service, args=(service_conf, ctrl_addrs[i], req_addrs[i], i))
    proc.start()
    procs.append(proc)

In [ ]:
def fit_events_nn(events, index, Truths, ctrl_addrs):
    outputs = []

    client = LLHClient(ctrl_addr=ctrl_addrs[index], conf_timeout=60000)
    def Eval_llh(params, event, fix=[None]):
        if fix[0] != None:
            params = np.insert(params, fix[0], fix[1])
            
        if ~np.alltrue(np.logical_and(bounds[:,0] <= params, params <= bounds[:,1]), axis=-1):
            return 1e9
        
        #event_red, remove_indx = remove_strings(event, [0,2])
        #c_data, _ = NNs.get_dom_data(np.array([event_red]), np.ones((1,8)), np.delete(detector, remove_indx, axis=0))
        c_data, _ = NNs.get_dom_data(np.array([event]), np.ones((1,8)), detector)
        return client.eval_llh(event[0], c_data, params)

    for j, event in enumerate(events):
        def eval_LLH(params):
            if params.ndim == 1:
                return Eval_llh(params, event)
            else:
                o = []
                for p in params:
                    o.append(Eval_llh(p, event))
                return np.array(o)

        # seeding
        initial_points = init_points(event[0], 97) #, seed=Truths[j]
        
        #free fit
        fit_res = spherical_opt.spherical_opt(
            func=eval_LLH,
            method="CRS2",
            initial_points=initial_points,
            rand=np.random.default_rng(42),
            spherical_indices=[[4,5]],
            batch_size=12,
        )
        outputs.append(fit_res['x'])
        
    return outputs

In [ ]:
events_to_process = len(events)
pool_size = 200
evts_per_proc = int(math.ceil(events_to_process/pool_size))
evt_splits = [events[i*evts_per_proc:(i+1)*evts_per_proc] for i in range(pool_size)] #_red
true_splits = [truths[i*evts_per_proc:(i+1)*evts_per_proc] for i in range(pool_size)]
print(sum(len(l) for l in evt_splits))

gpu_inds = np.arange(pool_size) % n_gpus

fit_events_partial = partial(
        fit_events_nn,
        ctrl_addrs=ctrl_addrs
)

In [ ]:
%%time
# reconstruct with a worker pool; one LLH client per worker
with Pool(pool_size) as p:
    outs = p.starmap(fit_events_partial, zip(evt_splits, gpu_inds, true_splits))

all_outs = sum((out for out in outs), [])
all_outs = np.array(all_outs).reshape((events_to_process, 8))
recos_nn = np.array(all_outs)

In [ ]:
diff_nn = recos_nn - truths

In [ ]:
# kill all the services
import zmq
for proc, ctrl_addr in zip(procs, ctrl_addrs): 
    with zmq.Context.instance().socket(zmq.REQ) as ctrl_sock:
        ctrl_sock.connect(ctrl_addr)
        ctrl_sock.send_string("die")
        proc.join()

### plot

In [ ]:
par_names = ['x', 'y', 'z', 't', 'azi', 'zen', 'Ecscd', 'Etrck']

In [ ]:
fig = plt.figure(figsize=(25, 17))

for i in range(8):
    r = min(bounds[i][1] - bounds[i][0], 200)
    bins = np.linspace(-r/2, r/2, 50)
    
    plt.subplot(3,3,i+1)
    #plt.hist(diff_ana[:, i], bins, alpha=0.5, label='Analytic')
    #plt.hist(diff_ana_red[:, i], bins, alpha=0.5, label='Analytic reduced')
    #plt.hist(diff_nn[:, i], bins, alpha=0.5, label='NN')
    #plt.hist(diff_nn_red[:, i], bins, alpha=0.5, label='NN reduced')
    #plt.hist(recos_ana[:, i], 50, alpha=0.5, label='Analytic')
    plt.hist(recos_nn[:, i], 50, alpha=0.5, label='NN')
    if i == 2: plt.legend()
    plt.xlabel(par_names[i])
    
#plt.savefig('images/upgrade_reco.png', bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,9))
ax = fig.add_subplot(projection='3d')
ax.scatter(recos_nn[:, 0], recos_nn[:, 1], recos_nn[:, 2])